In [1]:
import pandas as pd
import re
import string
import csv
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk import word_tokenize
from spellchecker import SpellChecker

In [2]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [4]:
twitter=pd.read_csv(r"tweets.csv")

In [5]:
twitter

,Text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted
0,RT @TheCoachesSite: In the U fourth and final ...,False,0,NaN,4/6/2019 17:11,False,NaN,1.110000e+18,NaN,"<a href=""http://twitter.com/download/iphone"" r...",coachescounsel,1,True,False
1,RT @NewquayZoo: We've got ASAP some exciting n...,False,0,NaN,4/6/2019 17:11,False,NaN,1.110000e+18,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",brglilly,1,True,False
2,The outcome of the PRIOR CHESTNUTS FUNDRAISER ...,False,0,NaN,4/6/2019 17:11,True,NaN,1.110000e+18,NaN,"<a href=""https://about.twitter.com/products/tw...",tcsbybloomex,0,False,False
3,@anandmahindra @TCS New india and old india <U...,False,0,anandmahindra,4/6/2019 17:10,False,1.110000e+18,1.110000e+18,3.419795e+07,"<a href=""http://twitter.com/download/iphone"" r...",JavedKhjaak,0,False,False
4,TCS Developing Five Blockchain Platforms https...,False,0,NaN,4/6/2019 17:09,False,NaN,1.110000e+18,NaN,"<a href=""http://localhost.com"" rel=""nofollow"">...",devnullius,0,False,False
5,@anandmahindra @AvinashGoraksha @TCS Sir TCS h...,False,0,anandmahindra,4/6/2019 17:09,True,1.110000e+18,1.110000e+18,3.419795e+07,"<a href=""http://twitter.com/download/android"" ...",avadhutak5985,0,False,False
6,"Why Flipkart, Amazon, OYO, Paytm and not TCS, ...",False,0,NaN,4/6/2019 17:08,False,NaN,1.110000e+18,NaN,"<a href=""http://stockmarketindia.tumblr.com/"" ...",stocks_in,0,False,False
7,"Why Flipkart, Amazon, OYO, Paytm and never TCS...",False,0,NaN,4/6/2019 17:08,False,NaN,1.110000e+18,NaN,"<a href=""http://publicize.wp.com/"" rel=""nofoll...",TRENDINGHITS1,0,False,False
8,"@anandmahindra @TCS Sir, you are great to give...",False,0,anandmahindra,4/6/2019 17:06,False,1.110000e+18,1.110000e+18,3.419795e+07,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",SandeshShiva,0,False,False
9,Sab tcs kar rhi hi ASAP on official demand,False,0,DIPTAMANDAL6,4/6/2019 17:06,False,1.110000e+18,1.110000e+18,1.060000e+18,"<a href=""http://twitter.com/download/android"" ...",SinhaDeepanshu,0,False,False


## Preprocessing

In [6]:
def stopword():
    stop=stopwords.words("english")
    twitter["Text"]=twitter["Text"].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))#Stopword removal

In [7]:
def remove():
    twitter["Text"]=twitter["Text"].apply(lambda x:re.sub(r"http\S+",'', x))
    twitter['Text']=twitter['Text'].apply(lambda x:re.sub(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+",'',x))
    twitter['Text']=twitter['Text'].apply(lambda x:re.sub('@[^\s]+','',x))
    twitter['Text']=twitter['Text'].apply(lambda x:re.sub('#[^\s]+','',x))
    twitter['Text']=twitter['Text'].apply(lambda x:re.sub('\$\w*','',x))

In [8]:
stopword()
remove()

In [9]:
punct_num = '''!()-àÂ[]{};:\n,<>./?@#%^"&\*_~0123456789=\x92\x92\x96\x85+|'"'''
def punctuation(x):
    no_punct = ""
    for char in x:
            if char not in punct_num :
                no_punct = no_punct + char  
    return(no_punct)
twitter['Text']=twitter['Text'].apply(lambda x:punctuation(x))

In [ ]:
fileName = r"slang.txt"
accessMode = "r"
def slang_translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        with open(fileName, accessMode) as myCSVfile:
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                if _str.upper() == row[0]:
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    return(' '.join(user_string))
twitter['Text']=twitter['Text'].apply(lambda x:slang_translator(x))

## Don't mess with next cell

In [ ]:
spell = SpellChecker()
def spellcheck(x):
        correct={}
        wrong=spell.unknown(x.split())
        for t in wrong:
            correct[t]=spell.correction(t)
        for t in correct.keys():
            x= re.sub(t,correct[t],x,flags=re.IGNORECASE)
        return(x)
twitter['Text']=twitter['Text'].apply(lambda x:spellcheck(x))

In [ ]:
def polarity():
    pole=[]
    t=[]
    for line in twitter["Text"]:
        temp=TextBlob(line).sentiment.polarity
        t.append(temp)
        if(temp>0):
            pole.append('positive')
        elif(temp<0):
            pole.append('negative')
        else:
            pole.append('neutral')
    return(pole,t)
pole,temp=polarity()
twitter['Sentiment']=pole
twitter['Sentiment_score']=temp

In [ ]:
plt.hist(pole,histtype='bar',align='mid')

In [9]:
twitter['Sentiment']=pd.Categorical(twitter['Sentiment']).codes
twitter['Sentiment']=pd.Categorical(twitter['Sentiment']).rename_categories([-1,0,1])

In [14]:
twitter.shape

(4572, 16)

In [10]:
c=[]
for i in range(twitter.shape[0]):
        if (any(i.isalnum() for i in twitter['Text'][i])==False):
               c.append(i)
for i in c:
    twitter=twitter.drop(i)

In [16]:
twitter.shape

(4500, 16)

## Splitting

In [11]:
train,test=train_test_split(twitter,train_size=0.75)

C:\Users\pawan_300\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [12]:
def token_text(data):
    token=[]
    for i in range(data.shape[0]):
        token.append(word_tokenize(data["Text"].iloc[i]))
    return(token)
test_token=token_text(test)
train_token=token_text(train)

# Vocabulary 

In [19]:
wordlist=[]
def build_vocab(data):
    for i in data:
        for j in i:
            wordlist.append(j)
build_vocab(test_token)
build_vocab(train_token)
wordlist=set(wordlist)

## Bag of words

In [20]:
def bow(data):
    vectorizer = CountVectorizer(vocabulary=wordlist)
    features=vectorizer.fit_transform(data['Text']).todense()
    word_features=vectorizer.vocabulary_
    return(features)

In [21]:
x_train_bow=bow(train)
x_test_bow=bow(test)

## tf-idf

In [22]:
def tfidf(data):
    vectorizer = TfidfVectorizer(vocabulary=wordlist)
    matrix = vectorizer.fit_transform(data['Text']).todense()
    return(matrix)

In [23]:
x_train_tfidf=tfidf(train)
x_test_tfidf=tfidf(test)

# Naive Bayes Classifier 

In [24]:
y_test=test['Sentiment']
y_train=train['Sentiment']

In [23]:
clf1=BernoulliNB()
clf1.fit(x_train_bow,y_train)
clf2=BernoulliNB()
clf2.fit(x_train_bow,y_train)
def naive():
    pred_bow=clf1.predict(x_test_bow)
    pred_tfidf=clf2.predict(x_test_tfidf)
    return(pred_bow,pred_tfidf)
result_bow,result_tfidf=naive()
print("Using Bag of words")
print("Accuracy  :",accuracy_score(result_bow,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_bow))
print("Classification report :")
print(classification_report(y_test,result_bow))
print("Using Tfidf")
print("Accuracy  :",accuracy_score(result_tfidf,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_tfidf))
print("Classification report :")
print(classification_report(y_test,result_tfidf))

Using Bag of words
Accuracy  : 0.7688888888888888
Confusion matrix  :
[[  0  88  50]
 [  0 468  39]
 [  0  83 397]]
Classification report :
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       138
           0       0.73      0.92      0.82       507
           1       0.82      0.83      0.82       480

   micro avg       0.77      0.77      0.77      1125
   macro avg       0.52      0.58      0.55      1125
weighted avg       0.68      0.77      0.72      1125

Using Tfidf
Accuracy  : 0.7688888888888888
Confusion matrix  :
[[  0  88  50]
 [  0 468  39]
 [  0  83 397]]
Classification report :
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       138
           0       0.73      0.92      0.82       507
           1       0.82      0.83      0.82       480

   micro avg       0.77      0.77      0.77      1125
   macro avg       0.52      0.58      0.55      1125
weighted avg       0

C:\Users\pawan_300\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Support Vector Machine

In [51]:
def su_vm():
    clf1=SVC(kernel='linear', C=1, gamma=1)
    clf1.fit(x_train_bow,y_train)
    pred_bow=clf1.predict(x_test_bow)
    clf2=SVC(kernel='linear', C=1, gamma=1)
    clf2.fit(x_train_bow,y_train)
    pred_tfidf=clf2.predict(x_test_tfidf)
    return(pred_bow,pred_tfidf)
result_bow,result_tfidf=su_vm()
print("Using Bag of words")
print("Accuracy  :",accuracy_score(result_bow,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_bow))
print("Classification report :")
print(classification_report(y_test,result_bow))
print("Using Tfidf")
print("Accuracy  :",accuracy_score(result_tfidf,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_tfidf))
print("Classification report :")
print(classification_report(y_test,result_tfidf))


Using Bag of words
Accuracy  : 0.8631111111111112
Confusion matrix  :
[[ 81  38  13]
 [ 14 496  18]
 [ 16  55 394]]
Classification report :
              precision    recall  f1-score   support

          -1       0.73      0.61      0.67       132
           0       0.84      0.94      0.89       528
           1       0.93      0.85      0.89       465

   micro avg       0.86      0.86      0.86      1125
   macro avg       0.83      0.80      0.81      1125
weighted avg       0.86      0.86      0.86      1125

Using Tfidf
Accuracy  : 0.5644444444444444
Confusion matrix  :
[[  7 125   0]
 [  0 528   0]
 [  1 364 100]]
Classification report :
              precision    recall  f1-score   support

          -1       0.88      0.05      0.10       132
           0       0.52      1.00      0.68       528
           1       1.00      0.22      0.35       465

   micro avg       0.56      0.56      0.56      1125
   macro avg       0.80      0.42      0.38      1125
weighted avg       0

# Random Forest

In [52]:
def random_forest():
    rf1= RandomForestClassifier(n_estimators = 100, random_state = 0)
    rf1.fit(x_train_bow,y_train.astype('int64'))
    pred_bow=rf1.predict(x_test_bow)
    rf2= RandomForestClassifier(n_estimators = 100, random_state = 0)
    rf2.fit(x_train_tfidf,y_train.astype('int64'))
    pred_tfidf=rf2.predict(x_test_tfidf)
    return(pred_bow,pred_tfidf)
result_bow,result_tfidf=random_forest()
print("Using Bag of words")
print("Accuracy  :",accuracy_score(result_bow,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_bow))
print("Classification report :")
print(classification_report(y_test,result_bow))
print("Using Tfidf")
print("Accuracy  :",accuracy_score(result_tfidf,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_tfidf))
print("Classification report :")
print(classification_report(y_test,result_tfidf))

Using Bag of words
Accuracy  : 0.8595555555555555
Confusion matrix  :
[[ 62  53  17]
 [  0 520   8]
 [  6  74 385]]
Classification report :
              precision    recall  f1-score   support

          -1       0.91      0.47      0.62       132
           0       0.80      0.98      0.89       528
           1       0.94      0.83      0.88       465

   micro avg       0.86      0.86      0.86      1125
   macro avg       0.88      0.76      0.80      1125
weighted avg       0.87      0.86      0.85      1125

Using Tfidf
Accuracy  : 0.8515555555555555
Confusion matrix  :
[[ 64  50  18]
 [  0 511  17]
 [  3  79 383]]
Classification report :
              precision    recall  f1-score   support

          -1       0.96      0.48      0.64       132
           0       0.80      0.97      0.88       528
           1       0.92      0.82      0.87       465

   micro avg       0.85      0.85      0.85      1125
   macro avg       0.89      0.76      0.80      1125
weighted avg       0

## Logistic Regression

In [53]:
def logistic():
    lr1=LogisticRegression(solver='liblinear',multi_class='auto',C=100)
    lr1.fit(x_train_bow,y_train)
    pred_bow=lr1.predict(x_test_bow)
    lr2= LogisticRegression(solver='liblinear',multi_class='auto',C=100)
    lr2.fit(x_train_tfidf,y_train)
    pred_tfidf=lr2.predict(x_test_tfidf)
    return(pred_bow,pred_tfidf)
result_bow,result_tfidf=logistic()
print("Using Bag of words")
print("Accuracy  :",accuracy_score(result_bow,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_bow))
print("Classification report :")
print(classification_report(y_test,result_bow))
print("Using Tfidf")
print("Accuracy  :",accuracy_score(result_tfidf,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_tfidf))
print("Classification report :")
print(classification_report(y_test,result_tfidf))

Using Bag of words
Accuracy  : 0.864
Confusion matrix  :
[[ 76  42  14]
 [  7 502  19]
 [ 12  59 394]]
Classification report :
              precision    recall  f1-score   support

          -1       0.80      0.58      0.67       132
           0       0.83      0.95      0.89       528
           1       0.92      0.85      0.88       465

   micro avg       0.86      0.86      0.86      1125
   macro avg       0.85      0.79      0.81      1125
weighted avg       0.87      0.86      0.86      1125

Using Tfidf
Accuracy  : 0.8488888888888889
Confusion matrix  :
[[ 72  45  15]
 [  3 493  32]
 [ 11  64 390]]
Classification report :
              precision    recall  f1-score   support

          -1       0.84      0.55      0.66       132
           0       0.82      0.93      0.87       528
           1       0.89      0.84      0.86       465

   micro avg       0.85      0.85      0.85      1125
   macro avg       0.85      0.77      0.80      1125
weighted avg       0.85      0.85

In [24]:
def adaboost():
    adb1=AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators = 100, random_state = 0))
    adb1.fit(x_train_bow,y_train)
    pred_bow=adb1.predict(x_test_bow)
    adb2=AdaBoostClassifier(base_estimator=RandomForestClassifier(n_estimators = 100, random_state = 0))
    adb2.fit(x_train_tfidf,y_train)
    pred_tfidf=adb2.predict(x_test_tfidf)
    return(pred_bow,pred_tfidf)
result_bow,result_tfidf=adaboost()
print("Using Bag of words")
print("Accuracy  :",accuracy_score(result_bow,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_bow))
print("Classification report :")
print(classification_report(y_test,result_bow))
print("Using Tfidf")
print("Accuracy  :",accuracy_score(result_tfidf,y_test))
print("Confusion matrix  :")
print(confusion_matrix(y_test,result_tfidf))
print("Classification report :")
print(classification_report(y_test,result_tfidf))

Using Bag of words
Accuracy  : 0.7733333333333333
Confusion matrix  :
[[ 36  69  22]
 [  9 502  23]
 [  4 128 332]]
Classification report :
              precision    recall  f1-score   support

          -1       0.73      0.28      0.41       127
           0       0.72      0.94      0.81       534
           1       0.88      0.72      0.79       464

   micro avg       0.77      0.77      0.77      1125
   macro avg       0.78      0.65      0.67      1125
weighted avg       0.79      0.77      0.76      1125

Using Tfidf
Accuracy  : 0.7902222222222223
Confusion matrix  :
[[ 42  65  20]
 [  6 492  36]
 [  2 107 355]]
Classification report :
              precision    recall  f1-score   support

          -1       0.84      0.33      0.47       127
           0       0.74      0.92      0.82       534
           1       0.86      0.77      0.81       464

   micro avg       0.79      0.79      0.79      1125
   macro avg       0.81      0.67      0.70      1125
weighted avg       0

In [25]:
twitter.shape

(4500, 16)

In [25]:
l=[]
for i in range(0,twitter.shape[0],18):
    t=0
    for j in range(i,i+18):
        t+=twitter['Sentiment_score'].iloc[j]
    l.append(t)

In [26]:
stock=pd.read_csv(r"tcs stock.csv")

In [27]:
stock['collective sentiment']=l

In [28]:
l=[""]
for i in range(1,stock.shape[0]):
    l.append((stock['Close'][i]-stock['Close'][i-1])/stock['Close'][i])
stock['relative change']=l

In [29]:
l=[""]
for i in range(1,stock.shape[0]):
    l.append((stock['Open'][i]-stock['Open'][i-1])/stock['Open'][i])
stock['relative change_Open']=l

In [30]:
stock

,Date,Open,High,Low,Close,Adj Close,Volume,collective sentiment,relative change,relative change_Open
0,4/10/2018,1462.849976,1478.619995,1462.849976,1468.800049,1447.591064,1184532,2.142911,,
1,4/11/2018,1475.300049,1512.500000,1472.800049,1507.069946,1485.308350,3124416,3.604040,0.0253936,0.00843901
2,4/12/2018,1505.000000,1575.000000,1503.969971,1569.619995,1546.955322,6114360,3.370833,0.0398504,0.0197342
3,4/13/2018,1575.000000,1620.400024,1566.050049,1576.650024,1553.883789,8179118,0.369048,0.00445884,0.0444444
4,4/16/2018,1558.000000,1599.969971,1555.380005,1593.819946,1570.805786,6270274,3.777237,0.0107728,-0.0109114
5,4/17/2018,1593.819946,1597.469971,1573.599976,1583.300049,1560.437866,3301724,2.736364,-0.00664429,0.0224743
6,4/18/2018,1583.000000,1596.719971,1572.930054,1579.550049,1556.741943,2433066,3.511071,-0.00237409,-0.00683509
7,4/19/2018,1589.949951,1608.599976,1582.880005,1595.569946,1572.530518,4820952,4.069643,0.0100402,0.00437118
8,4/20/2018,1625.000000,1711.180054,1620.000000,1701.219971,1676.654907,18465662,2.761111,0.0621025,0.0215693
9,4/23/2018,1712.000000,1778.949951,1694.780029,1704.319946,1679.710327,15185910,2.505411,0.00181889,0.0508178


In [31]:
stock.to_csv(r"stock1.csv")

In [36]:
twitter['Text'][1]

"RT @NewquayZoo: We've got ASAP some exciting news...\r\r\r\n\r\r\r\nTo begin the countdown to our 50th Anniversary, we will be launching a very special competÂ\x85"

In [40]:
a.iloc[0]

Sentiment          0
Sentiment_score    0
Name: 0, dtype: object